In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import time
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
tf.random.set_seed(42)

def load_and_preprocess_data():
    # Load CIFAR-10 dataset
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    
    # Normalize pixel values
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    
    # Convert labels to one-hot encoding
    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)
    
    # Resize images for VGG16 and ResNet50 (both expect 224x224)
    x_train_resized = tf.image.resize(x_train, (224, 224))
    x_test_resized = tf.image.resize(x_test, (224, 224))
    
    return x_train_resized, y_train, x_test_resized, y_test

def create_model(base_model_type, input_shape):
    input_tensor = Input(shape=input_shape)
    
    if base_model_type == "vgg16":
        base_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
    else:
        base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)
    
    # Add new layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(10, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Freeze the pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False
        
    return model

def train_and_evaluate_model(model, model_name, x_train, y_train, x_test, y_test):
    # Compile model
    model.compile(optimizer=Adam(learning_rate=0.001),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    
    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
    # Record start time
    start_time = time.time()
    
    # Train model
    history = model.fit(x_train, y_train,
                       batch_size=32,
                       epochs=10,
                       validation_split=0.2,
                       callbacks=[early_stopping])
    
    # Record training time
    training_time = time.time() - start_time
    
    # Evaluate model
    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    
    # Get predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)
    
    # Generate classification report
    class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                  'dog', 'frog', 'horse', 'ship', 'truck']
    report = classification_report(y_test_classes, y_pred_classes, 
                                 target_names=class_names, output_dict=True)
    
    # Create confusion matrix
    cm = confusion_matrix(y_test_classes, y_pred_classes)
    
    return {
        'model_name': model_name,
        'history': history.history,
        'test_accuracy': test_accuracy,
        'training_time': training_time,
        'classification_report': report,
        'confusion_matrix': cm
    }

def plot_results(results):
    # Plot training history
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    for result in results:
        plt.plot(result['history']['accuracy'], label=f"{result['model_name']} (Train)")
        plt.plot(result['history']['val_accuracy'], label=f"{result['model_name']} (Val)")
    plt.title('Model Accuracy During Training')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    for result in results:
        plt.plot(result['history']['loss'], label=f"{result['model_name']} (Train)")
        plt.plot(result['history']['val_loss'], label=f"{result['model_name']} (Val)")
    plt.title('Model Loss During Training')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

def main():
    # Load and preprocess data
    print("Loading and preprocessing data...")
    x_train, y_train, x_test, y_test = load_and_preprocess_data()
    
    results = []
    
    # Train and evaluate VGG16
    print("\nTraining VGG16...")
    vgg16_model = create_model("vgg16", (224, 224, 3))
    vgg16_results = train_and_evaluate_model(vgg16_model, "VGG16", 
                                           x_train, y_train, x_test, y_test)
    results.append(vgg16_results)
    
    # Train and evaluate ResNet50
    print("\nTraining ResNet50...")
    resnet50_model = create_model("resnet50", (224, 224, 3))
    resnet50_results = train_and_evaluate_model(resnet50_model, "ResNet50", 
                                              x_train, y_train, x_test, y_test)
    results.append(resnet50_results)
    
    # Print comparison results
    print("\nModel Comparison Results:")
    for result in results:
        print(f"\n{result['model_name']}:")
        print(f"Test Accuracy: {result['test_accuracy']:.4f}")
        print(f"Training Time: {result['training_time']:.2f} seconds")
        print("\nClassification Report:")
        report = result['classification_report']
        print(f"Macro Avg F1-Score: {report['macro avg']['f1-score']:.4f}")
        print(f"Weighted Avg F1-Score: {report['weighted avg']['f1-score']:.4f}")
    
    # Plot results
    plot_results(results)

if __name__ == "__main__":
    main()

Loading and preprocessing data...
  3178496/170498071 ━━━━━━━━━━━━━━━━━━━━ 57:20 21us/step

KeyboardInterrupt: 